In [ ]:
# imports
from transformers import DistilBertModel, DistilBertTokenizer
import torch
import networkx as nx
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cpu


In [ ]:
model = DistilBertModel.from_pretrained('distilbert-base-uncased', output_attentions=True).to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# getting the data
import json
import random

sentences = []

with open('/content/multinli_1.0_dev_matched.jsonl', 'r', encoding='utf-8') as infile:
  for line in infile:
    sentences.append(json.loads(line)['sentence1'])

# getting the sentences that start with an uppercase letter, in order to avoid sampling bad quality/spoken data
sentences_upper = []
for item in sentences:
  if item[0].isupper():
    sentences_upper.append(item)

sentences = sentences_upper
print(len(sentences))

random.seed(42)
random_sample = random.sample(sentences, 1000)
sentences = random_sample

print(sentences[0])


# sentences = sentences[1000:]




7499
Bush the elder came of age when New England Republicans led the party, and patrician manners were boons to a Republican.


In [ ]:
# tokenization
from tqdm import tqdm

batch_size = 32  # Adjust based on system's memory and GPU capacity
tokenized_data = []

for i in tqdm(range(0, len(sentences), batch_size)):
    batch = sentences[i:i + batch_size]  # Select a batch of sentences
    tokenized_batch = tokenizer(batch, return_tensors='pt', padding=False, truncation=True).to(device)
    tokenized_data.extend(
        [{"sentence": s, "tokenized_data": {k: v[idx] for k, v in tokenized_batch.items()}}
         for idx, s in enumerate(batch)]
    )

# for item in sentences:
#   tokenized_data.append({"sentence": item, "tokenized_data": tokenizer(item, return_tensors='pt').to(device)})
#   print(tokenized_data)


  0%|          | 0/32 [00:00<?, ?it/s]


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
import json
import networkx as nx
from tqdm import tqdm
import torch

# Assuming tokenized_data and model are already defined
batch_size = 32  # Adjust as needed
output_file = "/content/drive/MyDrive/seminar_data_hagen/output_no_padding_random_1000.jsonl"

with open(output_file, "a") as jsonl_file:
    with torch.no_grad():  # Disable gradients for inference
        for i in tqdm(range(0, len(tokenized_data), batch_size)):
            batch = tokenized_data[i:i + batch_size]

            # Combine tokenized inputs into a single batch
            inputs = {
                key: torch.stack([entry["tokenized_data"][key] for entry in batch]).to(device)
                for key in batch[0]["tokenized_data"]
            }

            # Pass inputs through the model
            outputs = model(**inputs)
            attentions = outputs.attentions  # Shape: (num_layers, batch_size, num_heads, seq_len, seq_len)

            num_layers = model.config.num_hidden_layers
            num_heads = model.config.num_attention_heads

            # Process attention scores layer by layer for each batch
            for layer_count in range(num_layers):
                for batch_idx, attention_scores in enumerate(attentions[layer_count]):
                    sentence = batch[batch_idx]["sentence"]  # Retrieve original sentence
                    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][batch_idx])
                    avg_attention = attention_scores.mean(dim=0)  # Average over heads, shape: (seq_len, seq_len)

                    # Build graph data
                    graph_data = {"nodes": [], "edges": []}
                    for i, token in enumerate(tokens):
                        graph_data["nodes"].append({"id": i, "label": token})

                    for i in range(len(tokens)):
                        for j in range(len(tokens)):
                            weight = avg_attention[i, j].item()
                            if weight > 0:  # Optionally set a threshold
                                graph_data["edges"].append({"source": i, "target": j, "weight": weight})

                    # Prepare JSON object for saving
                    data = {
                        "sentence": sentence,  # Include original sentence
                        "layer": layer_count,
                        "batch_index": batch_idx,
                        "tokens": tokens,
                        "graph": graph_data,
                    }

                    # Write to JSONL file
                    jsonl_file.write(json.dumps(data) + "\n")


In [ ]:
"""# Pass inputs through the model to get attention scores
with torch.no_grad():
    outputs = model(**inputs)
    attentions = outputs.attentions

num_layers = model.config.n_layers
print("Number of layers: ", num_layers)

num_heads = model.config.n_heads
print("Number of heads: ", num_heads)

# Extract attention scores layer by layer

for layer_count in range(num_layers):
  attention_scores = attentions[layer_count][0]  # Shape: (num_heads, seq_len, seq_len)
  avg_attention = attention_scores.mean(dim=0)

  G = nx.DiGraph()
  tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
  # Add nodes for each token
  for i, token in enumerate(tokens):
      G.add_node(i, label=token)

  # Add edges with weights as attention scores
  for i in range(len(tokens)):
      for j in range(len(tokens)):
          weight = avg_attention[i, j].item()
          if weight > 0:  # Optionally set a threshold for visualization
              G.add_edge(i, j, weight=weight)

  # Plot the graph
  plt.figure(figsize=(10, 10))
  pos = nx.spring_layout(G, seed=42)
  edge_labels = {(i, j): f"{G[i][j]['weight']:.2f}" for i, j in G.edges()}
  nx.draw(G, pos, with_labels=True, labels=nx.get_node_attributes(G, 'label'), node_size=700)
  nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
  plt.title(f"DistilBERT Self-Attention Graph (Layer {layer_count})")
  plt.show()"""

